# Recommendation Dashboard

## Project Description:

The following codes generates a movie recommendation dashboard app that allows the user to rate three movies and receive top 10 movie recommendations.

The recommendation engine is powered by a Singular Value Decomposition (SVD) algorithm.

In [ ]:
# Import packages
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_auth

import numpy as np
import pandas as pd
from pickle import dump
from pickle import load
from scipy.spatial.distance import cosine

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools

# Set login credentials
USERNAME_PASSWORD_PAIRS = [['data','analyst']]

# establish app
app=dash.Dash(external_stylesheets = [dbc.themes.GRID])

# Set login credentials
auth = dash_auth.BasicAuth(app,USERNAME_PASSWORD_PAIRS)
server = app.server

# Import movie data
ratings = pd.read_csv('Data/Movie_Ratings.csv')

# Set dropdown options
search = ratings['movie'].unique()
search_options = []
for x in search:
    search_options.append({'label':x,'value':x})

# Set radio button options
radio_options = [{'label': i, 'value': i} for i in [0,1,2,3,4,5]]

# Set app layout
app.layout = html.Div([
    
    # App header
    html.Div([html.H1('Movie Recommender Dashboard',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})]),
    
    # App descriptive subtitle
    html.H2('Choose & Rate Your Movie',
            style = {'font-family':'Arial, Helvetica, sans-serif',
                     'textAlign':'center',
                     'padding-top':20,
                     'padding-bottom':10}),
    
    # Dropdown menue 1
    html.Div([dbc.Row([dbc.Col(dcc.Dropdown(id='search',
                                            options = search_options,
                                            value = 'Jurassic Park (1993)',
                                            style = {'textAlign':'center'})),
                     
                     # Dropdown menue 2
                     dbc.Col(dcc.Dropdown(id='search1',
                                          options = search_options,
                                          value = 'Godfather, The (1972)',
                                          style = {'textAlign':'center'})),
                     # Dropdown menue 3
                     dbc.Col(dcc.Dropdown(id = 'search2',
                                          options = search_options,
                                          value = 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
                                          style = {'textAlign':'center'}))])],
             
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'padding-left':20,
                      'padding-right':20}),
              
    # Radio buttons 1     
    html.Div([dbc.Row([dbc.Col(dcc.RadioItems(id = 'rating',
                                              options = radio_options,
                                              value = 5,
                                              labelStyle = {'display': 'inline-block'})),
                       
                       # Radio buttons 2
                       dbc.Col(dcc.RadioItems(id = 'rating1',
                                              options = radio_options,
                                              value = 5,
                                              labelStyle = {'display': 'inline-block'})),
                       
                       # Radio buttons 3
                       dbc.Col(dcc.RadioItems(id = 'rating2',
                                              options = radio_options,
                                              value = 5,
                                              labelStyle = {'display': 'inline-block'}))])],
             
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'textAlign':'center',
                      'padding-top':10}),
             
    # Bar chart displaying recommendation results
    html.Div([dcc.Graph(id = 'feature_graphic')],
             style = {'font-family':'Arial, Helvetica, sans-serif',
                      'padding-top':50,
                      'padding-right':120,
                      'padding-bottom':20,
                      'margin-left':50}),
    
    # Instructions
    html.Div([html.H1('Instructions',
                      style = {'padding':10,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'}),
              
              html.Div(html.P(["The dashboard displays movie recommendations based on the user's \
                               movie ratings. The following dropdown menus can be leveraged either by typing \
                               or scrolling for the movie you would like to rate. Below the dropdowns are \
                               radio buttons where you can rate your selection. Once all fields are filled \
                               out, the bar plot will fill into scaled strengths of the recommendation \
                               (top ten movies in descending order). The title of movies can be found on the \
                               y-axis of the plot."]),                  
                       style = {'padding':30,
                                'font-family':'Arial, Helvetica, sans-serif',
                                'line-height':30,
                                'textAlign':'center',
                                'fontSize':20})]),
    
    # Final ending block
    html.Div([html.H1('',
                      style = {'padding':30,
                               'margin':0,
                               'font-family':'Arial, Helvetica, sans-serif',
                               'background':'#1E90FF',
                               'color':'#FFFFFF',
                               'textAlign':'center'})])],
    style={'margin':0})

# App callback function
@app.callback(Output('feature_graphic','figure'),
              [Input('search','value'),
               Input('rating','value'),
               Input('search1','value'),
               Input('rating1','value'),
               Input('search2','value'),
               Input('rating2','value')])

# App recommendation function
def recommendation(search,rating,search1,rating1,search2,rating2):

    # Create recommendation function
    def recommendation (mov1, rating1, mov2, rating2, mov3, rating3):

        # Load algo as pickle file
        algo = load(open('SVD_Model.pkl', 'rb'))

        # Cosine distance between vectors calculation
        def cosine_distance(vector_a = np.array, vector_b = np.array):
            return cosine(vector_a, vector_b)

        # Retrieve vectors by movie name
        def get_vector_by_movie_name(movie_name, trained_model):
            movie_row_idx = trained_model.trainset._raw2inner_id_items[movie_name]
            return trained_model.qi[movie_row_idx]

        # Get vectors by movie name for three restaurants
        vector1 = get_vector_by_movie_name(mov1, algo)
        score1 = rating1
        vector2 = get_vector_by_movie_name(mov2, algo)
        score2 = rating2
        vector3 = get_vector_by_movie_name(mov3, algo)
        score3 = rating3

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen movies' vectors against all other movie vectors
        similarity_table1 = []
        for movie_name in algo.trainset._raw2inner_id_items.keys():
            movie_vector = get_vector_by_movie_name(movie_name, algo)
            similarity_score = cosine_distance(vector1, movie_vector)
            similarity_table1.append((((1-similarity_score)), movie_name))

        # Convert similarity table into a data frame
        mov_rec1 = pd.DataFrame(similarity_table1, columns = ['similarity', 'movie name'])
        # Scale cosine score by rating
        mov_rec1['similarity'] = mov_rec1['similarity'] * score1
        # Sort data set to descending
        mov_rec1 = mov_rec1.sort_values('similarity', ascending = False)

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen movies' vectors against all other movie vectors
        similarity_table2 = []
        for movie_name in algo.trainset._raw2inner_id_items.keys():
            movie_vector = get_vector_by_movie_name(movie_name, algo)
            similarity_score = cosine_distance(vector2, movie_vector)
            similarity_table2.append((((1-similarity_score)), movie_name))

        # Convert similarity table into a data frame
        mov_rec2 = pd.DataFrame(similarity_table2, columns = ['similarity', 'movie name'])
        # Scale cosine score by rating
        mov_rec2['similarity'] = mov_rec2['similarity'] * score2
        # Sort data set to descending
        mov_rec2 = mov_rec2.sort_values('similarity', ascending = False)

        ##############################################################################################################

        # Calculate cosine similarity for all three chosen movies' vectors against all other movie vectors
        similarity_table3 = []
        for movie_name in algo.trainset._raw2inner_id_items.keys():
            movie_vector = get_vector_by_movie_name(movie_name, algo)
            similarity_score = cosine_distance(vector3, movie_vector)
            similarity_table3.append((((1-similarity_score)), movie_name))

        # Convert similarity table into a data frame
        mov_rec3 = pd.DataFrame(similarity_table3, columns = ['similarity', 'movie name'])
        # Scale cosine score by rating
        mov_rec3['similarity'] = mov_rec3['similarity'] * score3
        # Sort data set to descending
        mov_rec3 = mov_rec3.sort_values('similarity', ascending = False)
        
        ##############################################################################################################

        # Create a list of all data frames
        df_list = [mov_rec1, mov_rec2, mov_rec3]
        # Concatenate all data frames by axis 0
        mov_rec4 = pd.concat(df_list, axis = 0)
        # Remove all three chosen movies 
        mov_rec4 = mov_rec4.loc[(mov_rec4['movie name'] != mov1) & (mov_rec4['movie name'] != mov2) &
                                (mov_rec4['movie name'] != mov3)].reset_index(drop = True)
        # Scale cosine score by duplicates
        mov_rec4 = mov_rec4.groupby(by = "movie name").sum().reset_index()
        # Sort values by cosine values in descending order
        mov_rec4 = mov_rec4.sort_values('similarity', ascending = False).reset_index(drop = True)

        # Print recommendations
        print('\n')
        mov_rec4.info()
        return mov_rec4.head(10)

    # Data results from recommendation engine
    data = recommendation(search, rating, search1, rating1, search2, rating2)
    
    # Turn recommmendation data into lists to be processed by bar chart
    x = data.iloc[:10,1].values
    x = x[::-1]
    y = data.iloc[:10,0].values
    y = y[::-1]

    # Bar chart 
    data = [go.Bar(x=x,
                 y=y,
                 marker={'color':'#1E90FF'},
                 name='Movies',
                 orientation='h')]

    return {'data': data,
          'layout': go.Layout(margin={'l': 300, 'r': 40, 't': 40, 'b': 40})}


if __name__ == '__main__':
    app.run_server()